In [1]:
from produto import Produto
from caixa import Caixa
from estoque import Estoque
from onda import Onda

from pyspark.sql import SparkSession
from pyspark import SparkConf

# Configuração inicial do PySpark
conf = SparkConf()
conf.setAppName("MeuAplicativoPySpark")
conf.setMaster("local[*]")  # Usar todos os núcleos locais para execução

# Criação da SparkSession
spark = SparkSession.builder \
    .config(conf=conf) \
    .getOrCreate()

In [2]:
# Verificar se a sessão Spark está funcionando
print("SparkSession Criada!")
spark.sparkContext.setLogLevel("WARN")  # Define o nível de log para evitar muitas mensagens de depuração

# Leitura de dois arquivos CSV e salvando em variáveis
caixas_df = spark.read.csv("../data/caixas.csv", header=True, inferSchema=True)
estoque_df = spark.read.csv("../data/estoque.csv", header=True, inferSchema=True)

# Mostrando o conteúdo dos DataFrames
caixas_df.show(1)
estoque_df.show(1)


SparkSession Criada!
+-------+--------+-----+-------------+-----+
|ONDA_ID|CAIXA_ID|PECAS|  CLASSE_ONDA|  SKU|
+-------+--------+-----+-------------+-----+
|      4|      12|    1|CLASSE_ONDA_1|SKU_1|
+-------+--------+-----+-------------+-----+
only showing top 1 row

+-----+--------+---------+-----+
|ANDAR|CORREDOR|      SKU|PECAS|
+-----+--------+---------+-----+
|    0|       2|SKU_17028|  193|
+-----+--------+---------+-----+
only showing top 1 row



In [3]:
# Criando uma visão temporária para utilizar SQL puro
caixas_df.createOrReplaceTempView("caixas")
estoque_df.createOrReplaceTempView("estoque")

In [4]:

# Agregando as quantidades de cada SKU por classe de onda e caixa
agregado_caixa = spark.sql("""
    SELECT
        CLASSE_ONDA,
        CAIXA_ID,
        SKU,
        PECAS
    FROM caixas

""")
agregado_caixa.createOrReplaceTempView("agregado_caixa")

# Agregando as quantidades de cada SKU por andar
agregado_estoque = spark.sql("""
    SELECT
        ANDAR,
        SKU,
        SUM(PECAS) AS TOTAL_PECAS_DISPONIVEIS
    FROM estoque
    GROUP BY
        ANDAR, SKU
""")
agregado_estoque.createOrReplaceTempView("agregado_estoque")

# Verificando quantas peças são necessárias em cada caixa e quantas temos no andar
dados_necessarios_disponiveis = spark.sql("""
    SELECT
        ac.CLASSE_ONDA,
        ac.CAIXA_ID,
        ae.ANDAR,
        ac.SKU,
        ac.PECAS AS TOTAL_PECAS_NECESSARIAS,
        ae.TOTAL_PECAS_DISPONIVEIS,
        (ae.TOTAL_PECAS_DISPONIVEIS - ac.PECAS) AS DIFF
        
    FROM agregado_caixa ac
    LEFT JOIN agregado_estoque ae
    ON ac.SKU = ae.SKU
    ORDER BY
        ac.CLASSE_ONDA, ac.CAIXA_ID, ae.ANDAR
""")
dados_necessarios_disponiveis.show()

+-------------+--------+-----+------+-----------------------+-----------------------+----+
|  CLASSE_ONDA|CAIXA_ID|ANDAR|   SKU|TOTAL_PECAS_NECESSARIAS|TOTAL_PECAS_DISPONIVEIS|DIFF|
+-------------+--------+-----+------+-----------------------+-----------------------+----+
|CLASSE_ONDA_1|      12|    0| SKU_2|                      1|                     84|  83|
|CLASSE_ONDA_1|      12|    0| SKU_3|                      1|                    852| 851|
|CLASSE_ONDA_1|      12|    0| SKU_4|                      1|                    670| 669|
|CLASSE_ONDA_1|      12|    0| SKU_6|                      4|                    233| 229|
|CLASSE_ONDA_1|      12|    0| SKU_9|                      4|                    152| 148|
|CLASSE_ONDA_1|      12|    0|SKU_11|                      1|                      1|   0|
|CLASSE_ONDA_1|      12|    0|SKU_12|                      2|                    132| 130|
|CLASSE_ONDA_1|      12|    0|SKU_14|                      8|                    271| 263|

In [5]:
# Filtrando os casos onde o DIFF > 0
caixas_easy = dados_necessarios_disponiveis.filter("DIFF >= 0")
caixas_easy.count()

23299

In [6]:
caixas_hard = dados_necessarios_disponiveis.filter("DIFF < 0")
caixas_hard.count()

218

In [7]:
produtos = {}

for row in caixas_df.collect():
    sku = row['SKU']
    qtd = row['PECAS']
    if sku not in produtos:
        produtos[sku] = Produto(sku, qtd)
    else:
        produtos[sku].qtd += qtd 

print(len(produtos))

2085


In [8]:
caixas = {}
for row in caixas_df.collect():
    onda_id = row['ONDA_ID']
    caixa_id = row['CAIXA_ID']
    classe_onda = row['CLASSE_ONDA']
    sku = row['SKU']
    qtd = row['PECAS']
    
    # Criar ou buscar a caixa existente
    if caixa_id not in caixas:
        caixa = Caixa(caixa_id, classe_onda, onda_id)
        caixas[caixa_id] = caixa
    else:
        caixa = caixas[caixa_id]
    
    # Adicionar o produto à caixa
    produto = Produto(sku, qtd)
    caixa.add_produto(produto)

print(len(caixas))

Produto SKU_1 adicionado na caixa 4
Produto SKU_2 adicionado na caixa 4
Produto SKU_3 adicionado na caixa 4
Produto SKU_4 adicionado na caixa 4
Produto SKU_5 adicionado na caixa 4
Produto SKU_6 adicionado na caixa 4
Produto SKU_7 adicionado na caixa 4
Produto SKU_8 adicionado na caixa 4
Produto SKU_9 adicionado na caixa 4
Produto SKU_10 adicionado na caixa 4
Produto SKU_11 adicionado na caixa 4
Produto SKU_12 adicionado na caixa 4
Produto SKU_13 adicionado na caixa 4
Produto SKU_14 adicionado na caixa 4
Produto SKU_15 adicionado na caixa 4
Produto SKU_16 adicionado na caixa 4
Produto SKU_17 adicionado na caixa 4
Produto SKU_18 adicionado na caixa 4
Produto SKU_19 adicionado na caixa 4
Produto SKU_20 adicionado na caixa 4
Produto SKU_21 adicionado na caixa 4
Produto SKU_22 adicionado na caixa 4
Produto SKU_23 adicionado na caixa 4
Produto SKU_24 adicionado na caixa 4
Produto SKU_25 adicionado na caixa 4
Produto SKU_26 adicionado na caixa 4
Produto SKU_27 adicionado na caixa 4
Produto SK

In [9]:
estoques = []
for row in estoque_df.collect():
    andar = row['ANDAR']
    corredor = row['CORREDOR']
    sku = row['SKU']
    qtd = row['PECAS']
    
    produto = produtos.get(sku, Produto(sku, 0))
    estoque = Estoque(andar, corredor, produto, qtd)
    estoques.append(estoque)

print(len(estoques))

7798


In [13]:
for caixa in caixas.values():

    #print(f"Caixa da onda {caixa.id_onda} com {len(caixa.produtos)} produtos")

    andares = []
    fileiras = []

    for produto in caixa.produtos:
        for estoque in estoques:

            achou = 0
            
            if produto.sku == estoque.produto.sku and produto.qtd <= estoque.qtd:

                if estoque.andar not in andares:
                    andares.append(estoque.andar)

                if (estoque.corredor, estoque.andar) not in fileiras:
                    fileiras.append((estoque.corredor, estoque.andar))

                estoque.qtd -= produto.qtd

                achou = 1
                break

        if achou == 0:
            print(f"Produto {produto.sku} da caixa {caixa.id_caixa} não encontrado no estoque")

                #print(f"Produto {produto.sku} encontrado no estoque do andar {estoque.andar} e fileira {estoque.corredor}")


    print(f"Caixa {caixa.id_caixa} da onda {caixa.id_onda} com {len(caixa.produtos)} produtos nos andares {andares}")

Caixa 12 da onda 4 com 17 produtos nos andares [2, 0, 1]
Caixa 13 da onda 4 com 31 produtos nos andares [0, 2, 1]
Caixa 45 da onda 4 com 27 produtos nos andares [1, 0, 2]
Caixa 46 da onda 4 com 26 produtos nos andares [2, 1, 0]
Caixa 48 da onda 4 com 25 produtos nos andares [0, 2, 1]
Caixa 49 da onda 4 com 19 produtos nos andares [2, 1, 0]
Produto SKU_155 da caixa 184 não encontrado no estoque
Caixa 184 da onda 4 com 10 produtos nos andares [0, 2, 1]
Produto SKU_156 da caixa 249 não encontrado no estoque
Produto SKU_157 da caixa 249 não encontrado no estoque
Produto SKU_158 da caixa 249 não encontrado no estoque
Produto SKU_159 da caixa 249 não encontrado no estoque
Produto SKU_160 da caixa 249 não encontrado no estoque
Produto SKU_162 da caixa 249 não encontrado no estoque
Produto SKU_163 da caixa 249 não encontrado no estoque
Produto SKU_164 da caixa 249 não encontrado no estoque
Produto SKU_165 da caixa 249 não encontrado no estoque
Produto SKU_166 da caixa 249 não encontrado no est